<a href="https://colab.research.google.com/github/tarakantaacharya/NLPinternal/blob/main/rag_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
! pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.3 MB/s eta 0:00:00


In [6]:
! pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 24.5 MB/s eta 0:00:00


In [17]:
import torch
import torchaudio
from transformers import VitsModel, AutoTokenizer, AutoProcessor, AutoModelForCTC
from jiwer import wer, cer
from scipy.io.wavfile import write

# ✅ Load MMS-TTS Model (Odia)
tts_model = VitsModel.from_pretrained("facebook/mms-tts-ory")
tts_tokenizer = AutoTokenizer.from_pretrained("facebook/mms-tts-ory")

# ✅ Input Text in Oriya
text = "ମୁଁ ଆମ ଜୀବନର ଅତି ଉତ୍ତମ ଦିନ ଦେଖିଛି।"

# ✅ Convert Text to Speech (TTS)
tts_inputs = tts_tokenizer(text, return_tensors="pt")
with torch.no_grad():
    waveform = tts_model(**tts_inputs).waveform.squeeze().cpu().numpy()

# ✅ Save Speech as WAV File
output_wav_file = "generated_speech.wav"
write(output_wav_file, 16000, waveform)  # Save at 16kHz
print(f"Speech saved at: {output_wav_file}")

# Load model directly
from transformers import AutoProcessor, AutoModelForPreTraining
asr_processor = AutoProcessor.from_pretrained("facebook/wav2vec2-large-xlsr-53")
asr_model = AutoModelForPreTraining.from_pretrained("facebook/wav2vec2-large-xlsr-53")

# ✅ Load & Preprocess Audio
def load_audio(file_path):
    speech, rate = torchaudio.load(file_path)  # Load audio
    resampler = torchaudio.transforms.Resample(orig_freq=rate, new_freq=16000)  # Convert to 16kHz
    speech = resampler(speech).squeeze().numpy()
    return speech

# ✅ Speech-to-Text (ASR)
speech = load_audio(output_wav_file)
inputs = asr_processor(speech, sampling_rate=16000, return_tensors="pt", padding=True)

with torch.no_grad():
    logits = asr_model(**inputs).logits

predicted_ids = torch.argmax(logits, dim=-1)
predicted_text = asr_processor.batch_decode(predicted_ids)[0]

# ✅ Compute WER & CER
wer_score = wer(text, predicted_text)
cer_score = cer(text, predicted_text)

# ✅ Display Results
print(f"\n🔹 Original Text: {text}")
print(f"🔹 Predicted Text: {predicted_text}")
print(f"✅ Word Error Rate (WER): {wer_score:.4f}")
print(f"✅ Character Error Rate (CER): {cer_score:.4f}")

Speech saved at: generated_speech.wav


/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:61: FutureWarning: Loading a tokenizer inside Wav2Vec2Processor from a config that does not include a `tokenizer_class` attribute is deprecated and will be removed in v5. Please add `'tokenizer_class': 'Wav2Vec2CTCTokenizer'` attribute to either your `config.json` or `tokenizer_config.json` file to suppress this warning: 
  warnings.warn(


OSError: Can't load tokenizer for 'facebook/wav2vec2-large-xlsr-53'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'facebook/wav2vec2-large-xlsr-53' is the correct path to a directory containing all relevant files for a Wav2Vec2CTCTokenizer tokenizer.

In [ ]:
! pip install groq

In [ ]:
! pip install langchain
! pip install langchain-community

In [ ]:
! pip install chromadb

In [ ]:
! pip install faiss-cpu

In [ ]:
import pandas as pd
df_ap = pd.read_csv('/content/ap_eapcet-last-rank-2023.csv')
df_ts = pd.read_csv('/content/TSEAMCET_2023_FINALPHASE.csv')

In [ ]:
df_ap['State'] = "Andhra Pradesh"
df_ts['State'] = "Telangana"

In [ ]:
df_ts.columns

Index(['Inst\n Code', 'Institute Name', 'Place', 'Dist \nCode', 'Co Education',
       'College Type', 'Year of Estab', 'Branch Code', 'Branch Name',
       'OC \nBOYS', 'OC \nGIRLS', 'BC_A \nBOYS', 'BC_A \nGIRLS', 'BC_B \nBOYS',
       'BC_B \nGIRLS', 'BC_C \nBOYS', 'BC_C \nGIRLS', 'BC_D \nBOYS',
       'BC_D \nGIRLS', 'BC_E \nBOYS', 'BC_E \nGIRLS', 'SC \nBOYS',
       'SC \nGIRLS', 'ST \nBOYS', 'ST \nGIRLS', 'EWS \nGEN OU',
       'EWS \nGIRLS OU', 'Tuition Fee', 'Affiliated To', 'State'],
      dtype='object')

In [ ]:
df_ap.columns

Index(['SNO', 'INSTCODE', 'NAME OF THE INSTITUTION', 'TYPE', 'INST\r_RE\rG',
       'DIST', 'PLACE', 'COED', 'AFFL.', 'ESTD', 'A_RE\rG', 'branch_\rcode',
       'OC_BO\rYS', 'OC_GIR\rLS', 'SC_BO\rYS', 'SC_GIR\rLS', 'ST_BO\rYS',
       'ST_GIR\rLS', 'BCA_B\rOYS', 'BCA_GI\rRLS', 'BCB_B\rOYS', 'BCB_GI\rRLS',
       'BCC_B\rOYS', 'BCC_GI\rRLS', 'BCD_B\rOYS', 'BCD_GI\rRLS', 'BCE_B\rOYS',
       'BCE_GI\rRLS', 'OC_EWS_B\rOYS', 'OC_EWS_G\rIRLS', 'Unnamed: 30',
       'State'],
      dtype='object')

In [ ]:
df_ts.rename(columns={
    'Inst\n Code': 'Institute Code',
    'Institute Name': 'Institute Name',
    'College Type': 'College Type',
    'Place': 'Location',
    'Dist \nCode': 'District Code',
    'Co Education': 'Co-Ed Status',
    'Year of Estab': 'Established Year',
    'Branch Code': 'Branch Code',
    'Branch Name': 'Branch Name',
    'Tuition Fee': 'Tuition Fee',
    'Affiliated To': 'Affiliated To',
    'EWS \nGEN OU': 'EWS Boys',
    'EWS \nGIRLS OU': 'EWS Girls',
    'OC \nBOYS': 'OC Boys',
    'OC \nGIRLS': 'OC Girls',
    'SC \nBOYS': 'SC Boys',
    'SC \nGIRLS': 'SC Girls',
    'ST \nBOYS': 'ST Boys',
    'ST \nGIRLS': 'ST Girls',
    'BC_A \nBOYS': 'BC-A Boys',
    'BC_A \nGIRLS': 'BC-A Girls',
    'BC_E \nBOYS': 'BC-E Boys',
    'BC_E \nGIRLS': 'BC-E Girls'
}, inplace=True)

In [ ]:
df_ts.columns

Index(['Institute Code', 'Institute Name', 'Location', 'District Code',
       'Co-Ed Status', 'College Type', 'Established Year', 'Branch Code',
       'Branch Name', 'OC Boys', 'OC Girls', 'BC-A Boys', 'BC-A Girls',
       'BC_B \nBOYS', 'BC_B \nGIRLS', 'BC_C \nBOYS', 'BC_C \nGIRLS',
       'BC_D \nBOYS', 'BC_D \nGIRLS', 'BC-E Boys', 'BC-E Girls', 'SC Boys',
       'SC Girls', 'ST Boys', 'ST Girls', 'EWS Boys', 'EWS Girls',
       'Tuition Fee', 'Affiliated To', 'State'],
      dtype='object')

In [ ]:
df_ap.rename(columns={
    'INSTCODE': 'Institute Code',
    'NAME OF THE INSTITUTION': 'Institute Name',
    'TYPE': 'College Type',
    'PLACE': 'Location',
    'DIST': 'District Code',
    'COED': 'Co-Ed Status',
    'ESTD': 'Established Year',
    'branch_\rcode': 'Branch Code',
    'Branch Name': 'Branch Name',
    'Unnamed: 30': 'Tuition Fee',
    'AFFL.': 'Affiliated To',
    'OC_EWS_B\rOYS': 'EWS Boys',
    'OC_EWS_G\rIRLS': 'EWS Girls',
    'BCA_B\rOYS': 'BC-A Boys',
    'BCA_GI\rRLS': 'BC-A Girls',
    'BCE_B\rOYS': 'BC-E Boys',
    'BCE_GI\rRLS': 'BC-E Girls',
    'OC_BO\rYS': 'OC Boys',
    'OC_GIR\rLS': 'OC Girls',
    'SC_BO\rYS': 'SC Boys',
    'SC_GIR\rLS': 'SC Girls',
    'ST_BO\rYS': 'ST Boys',
    'ST_GIR\rLS': 'ST Girls',
}, inplace=True)

In [ ]:
df_ap.columns

Index(['SNO', 'Institute Code', 'Institute Name', 'College Type',
       'INST\r_RE\rG', 'District Code', 'Location', 'Co-Ed Status',
       'Affiliated To', 'Established Year', 'A_RE\rG', 'Branch Code',
       'OC Boys', 'OC Girls', 'SC Boys', 'SC Girls', 'ST Boys', 'ST Girls',
       'BC-A Boys', 'BC-A Girls', 'BCB_B\rOYS', 'BCB_GI\rRLS', 'BCC_B\rOYS',
       'BCC_GI\rRLS', 'BCD_B\rOYS', 'BCD_GI\rRLS', 'BC-E Boys', 'BC-E Girls',
       'EWS Boys', 'EWS Girls', 'Tuition Fee', 'State'],
      dtype='object')

In [ ]:
df_ts["Institution Registered"] = "AU"

In [ ]:
df_ap.rename(columns={"INST\r_RE\rG" : "Institution Registered"}, inplace=True)

In [ ]:
df_ap_new = df_ap.drop(columns=('A_RE\rG'))

In [ ]:
df_ts_new = df_ts.drop(columns=['BC_B \nBOYS', 'BC_B \nGIRLS', 'BC_C \nBOYS', 'BC_C \nGIRLS','BC_D \nBOYS', 'BC_D \nGIRLS'])

In [ ]:
df_ap_new = df_ap.drop(columns=['BCB_B\rOYS', 'BCB_GI\rRLS', 'BCC_B\rOYS','BCC_GI\rRLS', 'BCD_B\rOYS', 'BCD_GI\rRLS'])

In [ ]:
df_ts_new.columns

Index(['Institute Code', 'Institute Name', 'Location', 'District Code',
       'Co-Ed Status', 'College Type', 'Established Year', 'Branch Code',
       'Branch Name', 'OC Boys', 'OC Girls', 'BC-A Boys', 'BC-A Girls',
       'BC-E Boys', 'BC-E Girls', 'SC Boys', 'SC Girls', 'ST Boys', 'ST Girls',
       'EWS Boys', 'EWS Girls', 'Tuition Fee', 'Affiliated To', 'State',
       'Institution Registered'],
      dtype='object')

In [ ]:
df_ap_new.columns

Index(['SNO', 'Institute Code', 'Institute Name', 'College Type',
       'Institution Registered', 'District Code', 'Location', 'Co-Ed Status',
       'Affiliated To', 'Established Year', 'A_RE\rG', 'Branch Code',
       'OC Boys', 'OC Girls', 'SC Boys', 'SC Girls', 'ST Boys', 'ST Girls',
       'BC-A Boys', 'BC-A Girls', 'BC-E Boys', 'BC-E Girls', 'EWS Boys',
       'EWS Girls', 'Tuition Fee', 'State'],
      dtype='object')

In [ ]:
df_ap_new.drop(columns=['A_RE\rG'], inplace=True)

In [ ]:
df_ap_new.columns

Index(['SNO', 'Institute Code', 'Institute Name', 'College Type',
       'Institution Registered', 'District Code', 'Location', 'Co-Ed Status',
       'Affiliated To', 'Established Year', 'Branch Code', 'OC Boys',
       'OC Girls', 'SC Boys', 'SC Girls', 'ST Boys', 'ST Girls', 'BC-A Boys',
       'BC-A Girls', 'BC-E Boys', 'BC-E Girls', 'EWS Boys', 'EWS Girls',
       'Tuition Fee', 'State'],
      dtype='object')

In [ ]:
df_ts_new.columns

Index(['Institute Code', 'Institute Name', 'Location', 'District Code',
       'Co-Ed Status', 'College Type', 'Established Year', 'Branch Code',
       'Branch Name', 'OC Boys', 'OC Girls', 'BC-A Boys', 'BC-A Girls',
       'BC-E Boys', 'BC-E Girls', 'SC Boys', 'SC Girls', 'ST Boys', 'ST Girls',
       'EWS Boys', 'EWS Girls', 'Tuition Fee', 'Affiliated To', 'State',
       'Institution Registered'],
      dtype='object')

In [ ]:
df_ap_new.drop(columns=["SNO"], inplace=True)

In [ ]:
df_ap_new.columns

Index(['Institute Code', 'Institute Name', 'College Type',
       'Institution Registered', 'District Code', 'Location', 'Co-Ed Status',
       'Affiliated To', 'Established Year', 'Branch Code', 'OC Boys',
       'OC Girls', 'SC Boys', 'SC Girls', 'ST Boys', 'ST Girls', 'BC-A Boys',
       'BC-A Girls', 'BC-E Boys', 'BC-E Girls', 'EWS Boys', 'EWS Girls',
       'Tuition Fee', 'State'],
      dtype='object')

In [ ]:
df_ts_new.columns

Index(['Institute Code', 'Institute Name', 'Location', 'District Code',
       'Co-Ed Status', 'College Type', 'Established Year', 'Branch Code',
       'Branch Name', 'OC Boys', 'OC Girls', 'BC-A Boys', 'BC-A Girls',
       'BC-E Boys', 'BC-E Girls', 'SC Boys', 'SC Girls', 'ST Boys', 'ST Girls',
       'EWS Boys', 'EWS Girls', 'Tuition Fee', 'Affiliated To', 'State',
       'Institution Registered'],
      dtype='object')

In [ ]:
import pandas as pd
common_cols = list(set(df_ap_new.columns) & set(df_ts_new.columns))
df_combined = pd.concat([df_ap_new[common_cols], df_ts_new[common_cols]], ignore_index=True)

In [ ]:
df_combined.columns

Index(['State', 'BC-E Boys', 'SC Girls', 'Institute Code', 'Branch Code',
       'Tuition Fee', 'OC Girls', 'BC-A Girls', 'Location', 'OC Boys',
       'BC-A Boys', 'Institution Registered', 'Established Year', 'EWS Girls',
       'SC Boys', 'ST Girls', 'Affiliated To', 'ST Boys', 'Co-Ed Status',
       'EWS Boys', 'District Code', 'College Type', 'Institute Name',
       'BC-E Girls'],
      dtype='object')

In [ ]:
df_combined = df_combined.drop(columns=['BC-A Boys', 'BC-A Girls','SC Girls','ST Boys','BC-E Boys', 'EWS Boys', 'ST Girls','EWS Girls'])

In [ ]:
len(df_combined)

2530

In [ ]:
nan_counts = df_combined.isna().sum()
nan_counts

,0
State,0
Institute Code,0
Branch Code,0
Tuition Fee,59
OC Girls,1
Location,0
OC Boys,177
Institution Registered,0
Established Year,0
SC Boys,177


In [ ]:
df_combined.rename(columns={
    'OC Boys': 'Best Boy Rank',
    'OC Girls': 'Best Girl Rank',
}, inplace=True)

In [ ]:
df_combined.columns

Index(['State', 'Institute Code', 'Branch Code', 'Tuition Fee',
       'Best Girl Rank', 'Location', 'Best Boy Rank', 'Institution Registered',
       'Established Year', 'SC Boys', 'Affiliated To', 'Co-Ed Status',
       'District Code', 'College Type', 'Institute Name', 'BC-E Girls'],
      dtype='object')

In [ ]:
nan_counts = df_combined.isna().sum()
nan_counts

,0
State,0
Institute Code,0
Branch Code,0
Tuition Fee,59
Best Girl Rank,1
Location,0
Best Boy Rank,177
Institution Registered,0
Established Year,0
SC Boys,177


In [ ]:
df_combined['Institute Name'] = df_combined['Institute Name'].str.replace(r'[\r\n]', ' ', regex=True).str.strip()

In [ ]:
df_combined.to_csv('college_list.csv')

In [ ]:
grouped_df = df_combined.groupby('Institute Name').size().reset_index(name='Count')

print(grouped_df)

                                     Institute Name  Count
0            A C E ENGINEERING COLLEGE (AUTONOMOUS)     11
1              A M REDDY MEMORIAL COLL. OF PHARMACY      1
2                            A S N PHARMACY COLLEGE      1
3       A U COLLEGE OF ENGG FOR WOMEN- SELF FINANCE      6
4    A U COLLEGE OF ENGG-SELF FINANCE VISAKHAPATNAM      1
..                                              ...    ...
446           VMR PRADEEP KUMAR INST OF ENGG & TECH      1
447             WARANGAL INST OF TECHNOLOGY SCIENCE      1
448          WELLFARE INST OF SCIENCE TECH AND MGMT      5
449  WEST GODAVARI INSTT OF SCIENCE AND ENGINEERING      4
450                    YGVU YSR ENGINEERING COLLEGE      5

[451 rows x 2 columns]


In [ ]:
grouped_json = df_combined.groupby('Institute Name').apply(lambda x: x.drop(columns=['Institute Name']).to_dict(orient='records')).to_json()
# Save JSON to file
json_filename = "grouped_data.json"
with open(json_filename, "w") as f:
    f.write(grouped_json)

print(f"✅ JSON saved as {json_filename}")

✅ JSON saved as grouped_data.json


<ipython-input-47-349245edcf24>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_json = df_combined.groupby('Institute Name').apply(lambda x: x.drop(columns=['Institute Name']).to_dict(orient='records')).to_json()


In [ ]:
# Group and convert to dictionary
grouped_dict = df_combined.groupby('Institute Name', group_keys=False).apply(lambda x: x.drop(columns=['Institute Name']).to_dict(orient='records')).to_dict()

# Get the first category key and its data
first_category = list(grouped_dict.keys())[0]  # Get first category name
first_category_json = grouped_dict[first_category]  # Get first category data

print(f"First Category: {first_category}")
first_category_json

First Category: A C E ENGINEERING COLLEGE (AUTONOMOUS)


<ipython-input-50-c3a67596f5da>:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_dict = df_combined.groupby('Institute Name', group_keys=False).apply(lambda x: x.drop(columns=['Institute Name']).to_dict(orient='records')).to_dict()


[{'State': 'Telangana',
  'Institute Code': 'ACEG',
  'Branch Code': 'CSE',
  'Tuition Fee': 110000.0,
  'Best Girl Rank': 26405,
  'Location': 'GHATKESAR',
  'Best Boy Rank': 26405.0,
  'Institution Registered': 'AU',
  'Established Year': 2007,
  'SC Boys': 74408.0,
  'Affiliated To': 'JNTUH',
  'Co-Ed Status': 'COED',
  'District Code': 'MDL',
  'College Type': 'PVT',
  'BC-E Girls': 114390},
 {'State': 'Telangana',
  'Institute Code': 'ACEG',
  'Branch Code': 'INF',
  'Tuition Fee': 110000.0,
  'Best Girl Rank': 42711,
  'Location': 'GHATKESAR',
  'Best Boy Rank': 35401.0,
  'Institution Registered': 'AU',
  'Established Year': 2007,
  'SC Boys': 142386.0,
  'Affiliated To': 'JNTUH',
  'Co-Ed Status': 'COED',
  'District Code': 'MDL',
  'College Type': 'PVT',
  'BC-E Girls': 112713},
 {'State': 'Telangana',
  'Institute Code': 'ACEG',
  'Branch Code': 'AID',
  'Tuition Fee': 110000.0,
  'Best Girl Rank': 65375,
  'Location': 'GHATKESAR',
  'Best Boy Rank': 40102.0,
  'Institution R

In [ ]:
df_combined.to_csv('college_list.csv')

In [1]:
import pandas as pd
df_combined = pd.read_csv('college_list.csv')

In [2]:
df_combined.rename(columns={'Unnamed: 0': 'SNO'}, inplace=True)

In [3]:
len(df_combined)

450

In [ ]:
# Convert 'Best Rank' column to numeric before comparison
df_combined['Best Rank'] = pd.to_numeric(df_combined['Best Boy Rank'], errors='coerce')

# Now you can perform the comparison
if (df_combined['Best Rank'] < 5000).any():  # Use .any() to check if any value in the Series satisfies the condition
    # Since the condition applies to the entire Series, use boolean indexing to select the relevant rows
    filtered_df = df_combined[df_combined['Best Rank'] < 5000]
    print(filtered_df['Institute Name'])

96                        JNTUK COLLEGE OF ENGG. KAKINADA
97                        JNTUK COLLEGE OF ENGG. KAKINADA
705                    A U COLLEGE OF ENGG. VISAKHAPATNAM
706                    A U COLLEGE OF ENGG. VISAKHAPATNAM
760          GAYATHRI VIDYA PARISHAD COLL. OF ENGINEERING
761          GAYATHRI VIDYA PARISHAD COLL. OF ENGINEERING
1572    JNTUH UNIVERSITY COLLEGE OF ENGG SCI AND TECH ...
1573           CHAITANYA BHARATHI INSTITUTE OF TECHNOLOGY
1574      V N R VIGNANA JYOTHI INSTITUTE OF ENGG AND TECH
1575    JNTUH UNIVERSITY COLLEGE OF ENGG SCI AND TECH ...
1576           CHAITANYA BHARATHI INSTITUTE OF TECHNOLOGY
1577                       O U COLLEGE OF ENGG  HYDERABAD
1578      V N R VIGNANA JYOTHI INSTITUTE OF ENGG AND TECH
1579                        VASAVI COLLEGE OF ENGINEERING
1580    JNTUH UNIVERSITY COLLEGE OF ENGG SCI AND TECH ...
1581    JNTUH UNIVERSITY COLLEGE OF ENGG SCI AND TECH ...
1582           CHAITANYA BHARATHI INSTITUTE OF TECHNOLOGY
1583          

In [ ]:
import re
import groq  # Groq API for LLM

# Set up Groq API key (replace with your actual key)
groq.api_key = "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"  # Remove API key before sharing code publicly!

In [ ]:
df_combined.columns

Index(['SNO', 'State', 'Institute Code', 'Branch Code', 'Tuition Fee',
       'Best Girl Rank', 'Location', 'Best Boy Rank', 'Institution Registered',
       'Established Year', 'SC Boys', 'Affiliated To', 'Co-Ed Status',
       'District Code', 'College Type', 'Institute Name', 'BC-E Girls',
       'Best Rank'],
      dtype='object')

#RAG Workflow

---

In [4]:
! pip install chromadb

In [ ]:
import pandas as pd
import chromadb

# Load CSV file
df = pd.read_csv("/content/college_list.csv")

# Function to create a text column for each row
def generate_text(row):
    return (f"Institution: {row['Institute Name']}, located in {row['State']} ({row['Location']}), "
            f"Institute Code: {row['Institute Code']}, Branch Code: {row['Branch Code']}. "
            f"Tuition Fee: {row['Tuition Fee']}. Best Girl Rank: {row['Best Girl Rank']}, Best Boy Rank: {row['Best Boy Rank']}. "
            f"Established in {row['Established Year']}, affiliated to {row['Affiliated To']}. "
            f"College Type: {row['College Type']}, Co-Ed Status: {row['Co-Ed Status']}. "
            f"SC Boys Count: {row['SC Boys']}, Institution Registered: {row['Institution Registered']}.")

# Apply function to create 'text' column
df["text"] = df.apply(generate_text, axis=1)

# Save updated CSV
df.to_csv("updated_file.csv", index=False)

# ✅ Store in ChromaDB
client = chromadb.PersistentClient(path="./chroma_db")  # Specify DB path
collection = client.get_or_create_collection(name="college_data")

# Add data to ChromaDB
for index, row in df.iterrows():
    collection.add(
        ids=[str(index)],
        documents=[row["text"]]
    )

print("✅ Text column created & data stored in ChromaDB successfully!")

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:00<00:00, 105MiB/s]


✅ Text column created & data stored in ChromaDB successfully!


In [ ]:
import chromadb
from sentence_transformers import SentenceTransformer
import numpy as np

# ✅ Load ChromaDB
client = chromadb.PersistentClient(path="./chroma_db")
collection = client.get_or_create_collection(name="college_data")

# ✅ Retrieve all stored documents
retrieved_data = collection.get()  # Fetches all data

# ✅ Extract text and IDs
ids = retrieved_data["ids"]
documents = retrieved_data["documents"]

# ✅ Check if data exists
if not documents:
    print("❌ No documents found in ChromaDB!")
else:
    print(f"✅ Found {len(documents)} documents. Starting embedding...")

    # ✅ Load embedding model
    embedding_model = SentenceTransformer("all-MiniLM-L6-v2")  # Change if needed

    # ✅ Convert all text to embeddings
    embeddings = embedding_model.encode(documents, show_progress_bar=True)

    # ✅ Store embeddings back in ChromaDB
    for idx, embed in enumerate(embeddings):
        collection.update(
            ids=[ids[idx]],
            embeddings=[embed.tolist()]  # Convert NumPy array to list for storage
        )

    print("✅ All embeddings stored successfully!")

✅ Found 5060 documents. Starting embedding...


Batches:   0%|          | 0/159 [00:00<?, ?it/s]

✅ All embeddings stored successfully!


In [ ]:
import chromadb
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Initialize ChromaDB client
client = chromadb.PersistentClient(path="./chroma_db")
collection = client.get_or_create_collection(name="college_data")

# User query
query_text = "Which colleges in Andhra Pradesh have the best boy ranking?"

# Retrieve similar documents from ChromaDB
retrieved_docs = collection.query(
    query_texts=[query_text],
    n_results=300  # Retrieve top 300 similar documents
)

# Extract retrieved text
retrieved_texts = " ".join(retrieved_docs["documents"][0])

# Construct prompt for LLM
prompt = f"""You are an AI assistant that answers questions based on the retrieved context.
Use the following information to answer the query:
Context: {retrieved_texts}
Question: {query_text}
Answer:"""

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1", trust_remote_code=True)

# Load model with 4-bit quantization
model = AutoModelForCausalLM.from_pretrained(
    "deepseek-ai/DeepSeek-R1",
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16  # Use float16 for efficiency
)

# Tokenize input
inputs = tokenizer(prompt, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")

# Generate response
with torch.no_grad():
    output = model.generate(**inputs, max_length=200)

# Decode and print response
response_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("DeepSeek-R1 Response:", response_text)

The repository for deepseek-ai/DeepSeek-R1 contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/deepseek-ai/DeepSeek-R1.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] N


ValueError: Loading deepseek-ai/DeepSeek-R1 requires you to execute the configuration file in that repo on your local machine. Make sure you have read the code there to avoid malicious use, then set the option `trust_remote_code=True` to remove this error.

#End

---

In [ ]:
import json
import chromadb
from sentence_transformers import SentenceTransformer
from groq import Groq

# 🟢 Initialize ChromaDB client
client = chromadb.PersistentClient(path="./chroma_db")
collection = client.get_or_create_collection(name="colleges")

# 🟢 Load Sentence Transformer model for embeddings
model = SentenceTransformer("all-MiniLM-L6-v2")

# 🟢 Initialize Groq API client
groq_client = Groq(api_key="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")  # 🔴 Replace with your Groq API Key

# 🟢 User's query
query_text = "Top colleges with best CSE rank under 5000"

# Convert query to embedding
query_embedding = model.encode(query_text).tolist()

# Retrieve similar colleges from ChromaDB
results = collection.query(
    query_embeddings=[query_embedding],
    n_results=400,
    include=['metadatas']
)

# ✅ Process Results & Filter Based on CSE Rank
filtered_results = []
for metadata in results['metadatas'][0]:
    try:
        branches = json.loads(metadata['Branches'])  # Convert JSON string to list
    except TypeError:
        branches = metadata['Branches']  # If it's already a list

    for branch in branches:
        # Check if 'Best Boy Rank' is a dictionary and handle it
        best_boy_rank_value = branch.get('Best Boy Rank')
        if isinstance(best_boy_rank_value, dict):
            # If it's a dict, try to get the actual rank value from a key (e.g., 'rank')
            best_boy_rank_value = best_boy_rank_value.get('rank', float('inf'))  # Replace 'rank' with the actual key if needed

        try:
            best_boy_rank = int(best_boy_rank_value) if isinstance(best_boy_rank_value, (str, int, float)) and str(best_boy_rank_value).isdigit() else float('inf')  # Convert to int safely
        except ValueError:
            best_boy_rank = float('inf')

        if branch['Branch Code'] == 'CSE' and best_boy_rank < 5000:
            filtered_results.append(metadata)
            break  # Stop checking other branches if a match is found

# 🚨 **Strict RAG Check: If No Data, Return a Message**
if not filtered_results:
    print("❌ No colleges found that match your criteria.")
    exit()

# ✅ Format Retrieved Results
retrieved_info = "\n".join(
    [f"{i+1}. {r['Institute Name']} in {r['Location']} (Rank: {best_boy_rank})" for i, r in enumerate(filtered_results)]
)

# ✅ Create Final Prompt for Groq
final_prompt = f"""
User Query: {query_text}

Here are the top colleges for CSE with ranks under 5000:
{retrieved_info}

Generate a detailed response using ONLY this data. Do not add any extra information beyond what is listed.
"""

# 🟢 Pass to Groq API for response
response = groq_client.chat.completions.create(
    model="mixtral-8x7b-32768",
    messages=[
        {"role": "system", "content": "You are an assistant that answers strictly based on provided data."},
        {"role": "user", "content": final_prompt}
    ]
)

# 🟢 Print Groq Response
print("\n🔹 AI Response:\n")
print(response.choices[0].message.content)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



🔹 AI Response:

Based on the provided data, the top three colleges with a focus on Computer Science and Engineering (CSE) that have ranks under 5000 are:

1. A U COLLEGE OF ENGG. VISAKHAPATNAM, located in VISAKHAPATNAM with a rank of 108408.
2. GAYATHRI VIDYA PARISHAD COLL. OF ENGINEERING, also located in VISAKHAPATNAM, shares the same rank of 108408.
3. JNTUK COLLEGE OF ENGG. KAKINADA in KAKINADA holds the same rank of 108408.

Please note that while these colleges have a focus on CSE, they all share the same rank.


In [ ]:
import os
from groq import Groq

# 🔹 Set your API key
os.environ["GROQ_API_KEY"] = "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"

# 🔹 Initialize Groq client
client = Groq(api_key=os.environ["GROQ_API_KEY"])

In [ ]:
def retrieve_from_faiss(query, index, k=5):
    query_embedding = embedding_model.embed_query(query)
    query_embedding = np.array([query_embedding], dtype=np.float32)

    distances, indices = index.search(query_embedding, k)
    results = [texts[idx] for idx in indices[0] if idx < len(texts)]
    return results

In [ ]:
def retrieve_from_chromadb(query, collection, k=5):
    query_embedding = embedding_model.embed_query(query)
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=k
    )
    # Get metadatas from the first document in results
    metadatas = results['documents'][0]
    return [metadata for metadata in metadatas] # Return the list of metadatas

In [ ]:
def generate_answer(query, faiss_index, chroma_collection, k=5):
    # 🔹 Retrieve from FAISS
    faiss_results = retrieve_from_faiss(query, faiss_index, k)

    # 🔹 Retrieve from ChromaDB
    chroma_results = retrieve_from_chromadb(query, chroma_collection, k)

    # 🔹 Merge results (remove duplicates and None values)
    combined_results = list(set(result for result in faiss_results + chroma_results if result is not None)) # Filter out None values

    # 🔹 Create context from retrieved documents
    context = "\n".join(combined_results)

    # 🔹 Prepare prompt for LLM
    prompt = f"Use the stored data to answer the query and don't use any outside information:\n{context}\n\nQuery: {query}\nAnswer:"

    # 🔹 Call the LLM API (Groq or OpenAI)
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",  # Choose the best model
        messages=[{"role": "system", "content": "You are a helpful assistant."},
                  {"role": "user", "content": prompt}]
    )

    return response

In [ ]:
# Load FAISS index
faiss_index = faiss.read_index("vector_db/college_faiss_index.bin")

# Load ChromaDB collection
chroma_client = chromadb.PersistentClient(path="vector_db/college_chromadb")
chroma_collection = chroma_client.get_or_create_collection(name="colleges")

# Example Query
query = "Best engineering colleges in State Andhra Pradesh under rank 5000"
response = generate_answer(query, faiss_index, chroma_collection)

print("🤖 Chatbot Response:")
print(response.choices[0].message.content)

🤖 Chatbot Response:
Based on the stored data, the best engineering colleges in State Andhra Pradesh under rank 5000 are:

1. ADITYA ENGINEERING COLLEGE (Institute Code: ADTP) with Best Rank: 22560.0


---

---